In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from surrogate import ImageSurrogate

/home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-14 13:43:25.055479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 13:43:27.780156: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/mesa-diverted/x86_64-lin

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

In [4]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [5]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
'''import tensorflow as tf
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)
'''

'import tensorflow as tf\ntf.debugging.set_log_device_placement(True)\n\n# Create some tensors\na = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])\nb = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])\nc = tf.matmul(a, b)\n\nprint(c)\n'

## Load Data

In [7]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [8]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


2023-06-14 13:43:34.414064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 13:43:34.598753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 13:43:34.599607: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 13:43:34.604416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

### Batch Data

In [9]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [10]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20221113_03_18_29/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Train Surrogate

### Save Dir

In [12]:
date = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'surrogate'
model_dir = os.path.join(os.getcwd(), save_dir, date)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Train

In [13]:
surrogate = ImageSurrogate(model, model_dir)

t = time.time()
surrogate.train(train_data = ds_train, 
                val_data = ds_val, 
                max_epochs = 100, 
                batch_size = 32, 
                lookback = 20,
                lr = 1e-3)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)

Using alpha beta
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Epoch 1/100


2023-06-14 13:44:02.760490: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-06-14 13:44:08.006829: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fe2b40148c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-14 13:44:08.006881: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-06-14 13:44:08.054541: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-14 13:44:08.490743: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - ETA: 0s - loss: 0.8154 - accuracy: 0.7223
Epoch 1: val_loss improved from inf to 0.65349, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/surrogate/20230614_13_43_41/value_weights.h5
1563/1563 [==============================] - 733s 441ms/step - loss: 0.8154 - accuracy: 0.7223 - val_loss: 0.6535 - val_accuracy: 0.7898 - lr: 0.0010
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 0.4790 - accuracy: 0.8414
Epoch 2: val_loss improved from 0.65349 to 0.57889, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/surrogate/20230614_13_43_41/value_weights.h5
1563/1563 [==============================] - 692s 443ms/step - loss: 0.4790 - accuracy: 0.8414 - val_loss: 0.5789 - val_accuracy: 0.8082 - lr: 0.0010
Epoch 3/100
1563/1563 [==============================] - ETA: 0s - loss: 0.3539 - accuracy: 0.8837
Epoch 3: val_loss improved from 0.578

In [14]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]